In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from Loader import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import datetime

In [14]:
BATCH_SIZE = 128

x_train = None
x_val = None
x_train = DataLoader(BATCH_SIZE, 'training_generator')
x_val = DataLoader(BATCH_SIZE, 'training_generator', filename="val_chessData.csv")

In [15]:
lossHistory = []

class NnueCallbacks(tf.keras.callbacks.Callback):
  def on_batch_end(self, batch, logs=None):
    lossHistory[-1].append(logs['loss'])
    if batch and batch % 500 == 0:

      for i in lossHistory:
        plt.plot(i[20:], linewidth=0.5)
        plt.savefig("loss.png", dpi=400)
      plt.clf()

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [16]:
from keras import backend as K
def clipped_relu(x):
    return K.relu(x, max_value=4)

model = Sequential()
model.add(Dense(128, input_shape=(781,), activation=clipped_relu))
model.add(Dense(32, activation=clipped_relu))
model.add(Dense(32, activation=clipped_relu))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='mse', optimizer='adam', metrics=['mae'])

In [17]:
lossHistory.append([])
model.fit(x_train, validation_data=x_val, epochs=4, callbacks=[NnueCallbacks(), tensorboard_callback])

#Epoch 2/2
#10906/10906 [==============================] - 225s 21ms/step - loss: 0.0243 - mae: 0.0866

Epoch 1/4
24170/24170 [==============================] - 400s 17ms/step - loss: 0.0377 - mae: 0.1165 - val_loss: 0.0304 - val_mae: 0.1008
Epoch 2/4
24170/24170 [==============================] - 400s 17ms/step - loss: 0.0291 - mae: 0.0963 - val_loss: 0.0272 - val_mae: 0.0920
Epoch 3/4
24170/24170 [==============================] - 401s 17ms/step - loss: 0.0264 - mae: 0.0903 - val_loss: 0.0257 - val_mae: 0.0867
Epoch 4/4
24170/24170 [==============================] - 407s 17ms/step - loss: 0.0248 - mae: 0.0868 - val_loss: 0.0255 - val_mae: 0.0868


<Figure size 432x288 with 0 Axes>

In [11]:
import Features
import numpy as np

FEN = "r3n1k1/2qbbppp/4p3/3p2P1/2p2P2/2N1P3/Pr1B2BP/R2Q1RK1 b - - 2 19"
features = Features.get(FEN)
model.predict(np.array([features]))

array([[0.06667908]], dtype=float32)

In [22]:
model.save("production")

INFO:tensorflow:Assets written to: production/assets
